# Q Peak Detection Example (Empkins dataset)

## Setup and imports

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from fau_colors import cmaps

from pepbench.datasets import EmpkinsDataset
from pepbench.algorithms.ecg import QPeakExtractionVanLien2013
from pepbench.plotting.algorithms import plot_q_peak_extraction_vanlien2013

%matplotlib widget
%load_ext autoreload
%autoreload 2

## Plotting style

In [ ]:
plt.close("all")
palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", font="sans-serif", palette=palette)
plt.rcParams["figure.figsize"] = (10, 5)

## Point the dataset class to the local Empkins dataset folder inside example_data/
This cell auto-detects the repository root so the notebook works even if Jupyter's CWD isn't the repo root.

In [ ]:
empkins_base_path = "/Users/joana/Desktop/University/Master/Forschungspraktikum/Data/Empkins_Dataset"
# TODO: use example data

## Instantiate EmpkinsDataset
use_cache=False for reproducible load in this example

In [ ]:
dataset = EmpkinsDataset(empkins_base_path, use_cache=False, only_labeled=True)
dataset

### Show available participant/condition/phase rows and pick the first one

In [ ]:
print("Dataset index (first 10 rows):")
display(dataset.index.head(10))

## Select the first index entry
(participant, condition, phase)

In [ ]:
row = dataset.index.iloc[0]
participant, condition, phase = row["participant"], row["condition"], row["phase"]
print("Using:", participant, condition, phase)

## Create a subset for that single participant/condition/phase

In [ ]:
datapoint = dataset.get_subset(participant=participant, condition=condition, phase=phase)

## Run Q-peak extraction (Van Lien 2013)

In [ ]:
q_algo = QPeakExtractionVanLien2013()
# use the computed heartbeat segmentation
q_algo.extract(ecg=datapoint.ecg, heartbeats=datapoint.heartbeats, sampling_rate_hz=datapoint.sampling_rate_ecg)

## Inspect detected points (first rows)

In [ ]:
display(q_algo.points_.head())

## Plot a few heartbeats to visualise detected Q-peaks
determine the first three heartbeat_ids available in the reference_heartbeats index

In [ ]:
hb_ids = list(datapoint.reference_heartbeats.index.get_level_values("heartbeat_id").unique())[:3]
print("Plotting heartbeat ids:", hb_ids)
fig, ax = plot_q_peak_extraction_vanlien2013(datapoint, heartbeat_subset=hb_ids, normalize_time=True)
plt.show()
# TODO: try alternative algorithms:
# q_algo2 = QPeakExtractionMartinez2004Neurokit(); q_algo2.extract(ecg=datapoint.ecg, heartbeats=datapoint.heartbeats, sampling_rate_hz=datapoint.sampling_rate_ecg)
# fig2, ax2 = plot_q_peak_extraction_martinez2004_neurokit(datapoint, heartbeat_subset=hb_ids)

### Notes:
- You can swap `QPeakExtractionVanLien2013` for other algorithms available in `pepbench.algorithms.ecg` (e.g., `QPeakExtractionMartinez2004Neurokit`, `QPeakExtractionForouzanfar2018`).
- To use non-reference heartbeats, use `datapoint.heartbeats` instead of `datapoint.reference_heartbeats`.
- If the dataset is large, set `use_cache=True` when instantiating `EmpkinsDataset` for faster repeated loads.
